### Importing Dependencies

In [1]:
import numpy as np
import pandas as pd
import re
import nltk
#nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### Importing Twitter sentiment dataset

In [22]:
column_names = ['target', 'id', 'date', 'flag', 'user', 'text']
twitter_data = pd.read_csv('../twitter_data.csv', names=column_names, encoding = 'ISO-8859-1')
twitter_data.shape
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [23]:
twitter_data.isnull().sum()
twitter_data['target'].value_counts()

target
0    800000
4    800000
Name: count, dtype: int64

In [24]:
#convert target "4" to "1" which labels positive
twitter_data.replace( {'target' : {4:1}}, inplace=True)

##### 1 -> Positive Tweet and 0 -> Negative Tweet

In [2]:
def lemmatization(content):
    lemmatizer = WordNetLemmatizer()
    content = re.sub('[^a-zA-Z]', ' ', content)  # Remove non-alphabetic characters
    content = content.lower()  # Convert to lowercase
    tokens = content.split()  # Tokenize
    stop_words = set(stopwords.words("english"))
    tokens = [token for token in tokens if token not in stop_words]  # Remove stop words
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]  # Lemmatize
    return ' '.join(lemmatized_tokens)  # Return as a single string

In [ ]:
twitter_data['lemmatized_content'] = twitter_data['text'].apply(lemmatization)

In [2]:
twitter_data = pd.read_csv('../twitter_data_cleaned.csv', encoding = 'ISO-8859-1')
twitter_data.shape
twitter_data.head()

,target,id,date,flag,user,text,lemmatized_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset update facebook texting might cry result...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dived many time ball managed save res...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole body feel itchy like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behaving mad see


In [4]:
twitter_data['lemmatized_content'].head()

0    switchfoot http twitpic com zl awww bummer sho...
1    upset update facebook texting might cry result...
2    kenichan dived many time ball managed save res...
3                      whole body feel itchy like fire
4                     nationwideclass behaving mad see
Name: lemmatized_content, dtype: object

In [5]:
twitter_data['target'].head()

0    0
1    0
2    0
3    0
4    0
Name: target, dtype: int64

### Splitting of data to train and test

In [22]:
x = twitter_data['lemmatized_content']
y = twitter_data['target'].astype(str)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=2)
print(x_train.shape, x_test.shape)

(1280000,) (320000,)


In [23]:
x_train = x_train.fillna("")
x_test = x_test.fillna("")

### converting textual data into numerical data

In [24]:
vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [25]:
print(x_train)

  (0, 33739)	0.6715898890194907
  (0, 141607)	0.7409230870790761
  (1, 261187)	0.7706131833621019
  (1, 473471)	0.6373031630460713
  (2, 256060)	0.6346233363374544
  (2, 46210)	0.22864217950664856
  (2, 139734)	0.28075426335953335
  (2, 101506)	0.16556421427650894
  (2, 449676)	0.2976098191068681
  (2, 152980)	0.2229794547570607
  (2, 158891)	0.1745921250038019
  (2, 300173)	0.4859051815591137
  (2, 444025)	0.18402122712040125
  (3, 219349)	0.5308519458704057
  (3, 332765)	0.3898696744312958
  (3, 194863)	0.5254476376381264
  (3, 481971)	0.20664937198498243
  (3, 9322)	0.3556538194271432
  (3, 308762)	0.3477197526889277
  (4, 494341)	0.7443826049169564
  (4, 325886)	0.3668845332053293
  (4, 339736)	0.32217737288549236
  (4, 450796)	0.3210276191565379
  (4, 246084)	0.32316139146104883
  (5, 101506)	0.17745003043052615
  :	:
  (1279996, 144107)	0.26329030357945343
  (1279996, 488752)	0.5641448905590235
  (1279996, 37214)	0.4102104567351449
  (1279997, 454473)	0.264830240769143
  (1279997

In [26]:
print(x_test)

  (0, 6197)	0.2993950278814007
  (0, 57242)	0.38328434124780836
  (0, 163497)	0.12081665384673111
  (0, 193256)	0.12819419744172045
  (0, 238172)	0.12596658640334077
  (0, 293562)	0.3139488397046049
  (0, 302000)	0.4093901634043561
  (0, 321761)	0.13435039277622318
  (0, 337853)	0.3342014062650644
  (0, 342062)	0.18208687975436086
  (0, 360644)	0.18237652729459886
  (0, 401649)	0.20984159563879864
  (0, 418324)	0.16266526664693737
  (0, 447918)	0.1979807393208656
  (0, 462781)	0.3398058326543241
  (0, 482040)	0.1705234747434435
  (1, 32983)	0.5655611697739846
  (1, 168271)	0.8247063497050827
  (2, 42446)	0.30100379682696776
  (2, 46997)	0.3915226544626526
  (2, 330410)	0.5219720647077872
  (2, 372090)	0.5026123705406926
  (2, 415707)	0.48065860439861363
  (3, 62107)	0.7161182895481183
  (3, 236883)	0.482087739207169
  :	:
  (319996, 143921)	0.26737649917556927
  (319996, 147461)	0.29269435566220614
  (319996, 147530)	0.3080065607080351
  (319996, 351134)	0.39307456104492694
  (319996, 

### Training the Machine Learning model

### 1. Logistic Regression : classification based model

In [27]:
model = LogisticRegression(max_iter = 1000)

In [28]:
model.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

### Model Evaluation : accuracy_score

In [29]:
# accuracy_score on training data
y_train_pred = model.predict(x_train)
training_data_accuracy = accuracy_score(y_train, y_train_pred)
print("accuracy_score on training data : ", training_data_accuracy)

accuracy_score on training data :  0.81288046875


In [31]:
# accuracy_score on test data
y_test_pred = model.predict(x_test)
test_data_accuracy = accuracy_score(y_test, y_test_pred)
print("accuracy_score on test data : ", test_data_accuracy)

accuracy_score on test data :  0.7817375


### Saving the trained model

In [33]:
import pickle

In [34]:
filename = "trained_model.sav"
pickle.dump(model, open(filename, 'wb'))

### using the saved model for future predictions

In [37]:
#loading the saving model
loaded_model = pickle.load(open('../Twitter-Sentimental-Analysis/trained_model.sav', 'rb')) 

In [38]:
x_new = x_test[200]
print("actual value : ", y_test[200] )
y_new = loaded_model.predict(x_new)

if(y_new[0] == 0):
    print("Negative Tweet")
else:
    print("Positive Tweet")

actual value :  0
Positive Tweet


In [35]:
!git add .
!git commit -m "Saving the trained model using pickle"
!git push -u origin main

[main cfc891f] Saving the trained model using pickle
 3 files changed, 68 insertions(+), 10 deletions(-)
 create mode 100644 trained_model.sav
branch 'main' set up to track 'origin/main'.


To https://github.com/RevanasiddaNK/Twitter-Sentimental-Analysis.git
   c6c4489..cfc891f  main -> main
